<a href="https://colab.research.google.com/github/santiagorey16/PROGCOM-A/blob/main/Juego.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Juego

In [1]:
import random, time, csv, os, sys
from datetime import datetime

# ---- CONFIG ----
TARGET_SCORE = 100
TIME_LIMIT_SEC = 300  # 5 minutos
CSV_FILE = "bitacora_partidas.csv"

PALABRAS_TILDE = [
    ("camion", "camión"), ("arbol", "árbol"), ("bebe", "bebé"),
    ("facil", "fácil"), ("examen", "examen"), ("compas", "compás")
]
SINONIMOS = [
    ("feliz", ["contento","alegre"]), ("rápido", ["veloz","ligero"]),
    ("grande", ["enorme","gigante"]), ("comenzar", ["iniciar","empezar"])
]

def log_event(modulo, dificultad, correcto, t_resp, score_final):
    exists = os.path.exists(CSV_FILE)
    with open(CSV_FILE, "a", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        if not exists:
            w.writerow(["fecha","modulo","dificultad","correcto","tiempo_respuesta_s","puntaje_final"])
        w.writerow([datetime.now().isoformat(timespec="seconds"), modulo, dificultad, int(correcto), round(t_resp,2), score_final])

def gen_mate(dif):
    # dif 1: +/-, dif 2: x, dif 3: mixto hasta 12, dif 4: fracciones simples
    if dif < 4:
        a = random.randint(1, 9+3*dif)
        b = random.randint(1, 9+3*dif)
        if dif == 1:
            op = random.choice(["+","-"])
        elif dif == 2:
            op = random.choice(["+","-","*"])
        else:
            op = random.choice(["+","-","*"])
        exp = f"{a} {op} {b}"
        ans = eval(exp)
        return ("MATE", f"Resuelve: {exp} =", str(ans))
    else:
        # fracción equivalente simple
        n = random.randint(1, 9)
        d = random.randint(n+1, 12)
        k = random.randint(2, 4)
        enun = f"¿Cuál es equivalente a {n}/{d}? a) {n*k}/{d*k}  b) {n}/{d+k}  c) {n+k}/{d}"
        return ("MATE", enun, "a")

def gen_lengua(dif):
    t = random.choice(["tilde","bvs","sinonimo"])
    if t == "tilde":
        base, correcto = random.choice(PALABRAS_TILDE)
        texto = f"Escribe la forma con tilde correcta: {base}"
        return ("LENGUA", texto, correcto)
    elif t == "bvs":
        # regla b/v en prefijos "ob-/sub-" versus "va/ve/vi"
        palabra = random.choice(["obvio","subvenir","subvalorar","llevar","beber","votar"])
        # pregunta de opciones
        mal = {"obvio":"ovbio","subvenir":"subbenir","subvalorar":"subbalorar",
               "llevar":"llebar","beber":"veber","votar":"botar"}[palabra]
        opciones = [palabra, mal]
        random.shuffle(opciones)
        enun = f"Elige la forma correcta (a/b): a) {opciones[0]}   b) {opciones[1]}"
        correcta = "a" if opciones[0]==palabra else "b"
        return ("LENGUA", enun, correcta)
    else:
        base, lista = random.choice(SINONIMOS)
        correcto = random.choice(lista)
        return ("LENGUA", f"Escribe un sinónimo de '{base}'", correcto)

def preguntar(card, dificultad):
    modulo, enunciado, respuesta = card
    t0 = time.time()
    user = input(enunciado+" ").strip()
    t1 = time.time()
    correcto = False
    if respuesta.isdigit() or (respuesta.startswith("-") and respuesta[1:].isdigit()):
        # comparar numérica
        try:
            correcto = int(user) == int(respuesta)
        except:
            try:
                correcto = float(user) == float(respuesta)
            except:
                correcto = False
    else:
        correcto = user.lower() == respuesta.lower()
    return modulo, correcto, (t1 - t0), respuesta

def main():
    print("=== Misión Neurona: Aprende y Avanza ===")
    print("Objetivo: llega a 100 puntos en 5 minutos. ¡Suerte!\n")
    score, streak, dificultad = 0, 0, 1
    start = time.time()
    while time.time() - start < TIME_LIMIT_SEC and score < TARGET_SCORE:
        modulo = random.choice(["MATE","LENGUA"])
        card = gen_mate(dificultad) if modulo=="MATE" else gen_lengua(dificultad)
        mod, ok, t_resp, resp = preguntar(card, dificultad)
        if ok:
            streak += 1
            gain = 10 + 2*max(0, streak-2)
            score += gain
            print(f"✅ ¡Bien! +{gain} puntos | racha {streak} | total {score}")
            if streak >= 2 and dificultad < 4:
                dificultad += 1
        else:
            streak = 0
            score = max(0, score-5)
            print(f"❌ Incorrecto. Respuesta correcta: {resp} | −5 puntos | total {score}")
            if dificultad > 1:
                dificultad -= 1
        log_event(mod, dificultad, ok, t_resp, score)
        print("-"*50)
    # fin de partida
    dur = int(time.time()-start)
    if score >= TARGET_SCORE:
        print(f"🎉 ¡Misión cumplida! Puntaje {score} en {dur}s.")
    else:
        print(f"⏱️ Tiempo agotado. Puntaje final: {score} en {dur}s.")
    print(f"Bitácora guardada en: {os.path.abspath(CSV_FILE)}")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\nJuego interrumpido.")
        sys.exit(0)

=== Misión Neurona: Aprende y Avanza ===
Objetivo: llega a 100 puntos en 5 minutos. ¡Suerte!

Resuelve: 8 + 4 = 12
✅ ¡Bien! +10 puntos | racha 1 | total 10
--------------------------------------------------


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




Juego interrumpido.
Traceback (most recent call last):
  File "/tmp/ipython-input-1494877101.py", line 123, in <cell line: 0>
    main()
  File "/tmp/ipython-input-1494877101.py", line 97, in main
    mod, ok, t_resp, resp = preguntar(card, dificultad)
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1494877101.py", line 73, in preguntar
    user = input(enunciado+" ").strip()
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelbase.py", line 1177, in raw_input
    return self._input_request(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelbase.py", line 1219, in _input_request
    raise KeyboardInterrupt("Interrupted by user") from None
KeyboardInterrupt: Interrupted by user

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", 

TypeError: object of type 'NoneType' has no len()